In this notebook you will learn how to create an augmented simulator and how to train it. 

As always on these notebooks, we use the `Benchmark1` to demonstrate how to perform this task.

On the first section, we explain how to use a model that is already available on this reposotiry. The second section is dedicated to the explanation of what is needed to create a different kind of `AugmentedSimulator` with a different Neural Network archiecture with a customized loss etc.

# Load the benchmark dataset

As always the first step is always to load our dataset.

In [2]:
import os
from lips.neurips_benchmark import NeuripsBenchmark1
path_benchmark = os.path.join("reference_data")
neurips_benchmark1 = NeuripsBenchmark1(path_benchmark=path_benchmark,
                                       load_data_set=True)

## Train an available model

In this section we explain how to tune an available model. We take the example of the `FullyConnectedAS` that is an available fully connected neural network.

This section supposes that you already have a "model" (for example based on neural networks) that meets the `AugmentedSimulator` interface. If you do not have it already, the next section will cover the main principles.

**NB** The creation of the 'augmented_simulator' depends on each type of 'augmented_simulator'. 

The first step is to create the class you want to use, with the meta parameters you want to test. For this example:

In [3]:
path_save = os.path.join("trained_models")
if not os.path.exists(path_save):
    os.mkdir(path_save)

from tensorflow.keras.layers import Dense
from lips.augmented_simulators import FullyConnectedAS

# the three lines bellow might be familiar to the tensorflow users. They tell tensorflow to not take all
# the GPU video RAM for the model.
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for el in physical_devices:
    tf.config.experimental.set_memory_growth(el, True)

my_simulator = FullyConnectedAS(name="test_FullyConnectedAS",
                                # `attr_x` represents the variables of the dataset you want to use to predict 
                                # the output.
                                attr_x=("prod_p", "prod_v", "load_p", "load_q", "line_status", "topo_vect"),
                                # `attr_y` represents the variables of the dataset you want to predict
                                # we predict everything needed, you can try to change them if you want, add some 
                                # others etc.
                                attr_y=("a_or", "a_ex"),
                                # `sizes_layer` represents the size of each hidden layer in the neural network. The number
                                # of layers is determined by the length of this list, for example
                                sizes_layer=(300, 300, 300, 300),
                                # `lr` is the learning rate
                                lr=3e-4, 
                                # `layer` is the type of keras layer you want to use. We don't recommend to 
                                # change it
                                layer=Dense,
                                # `layer_act` is the activation function you want to use after each layer
                                layer_act="relu",
                                # `loss` is the training loss
                                loss="mse",  # loss used to train the model
                                # `batch_size` is the size of the batch for training
                                batch_size=128)

Then you need to train it. For example here we will train it for 200 epochs.

**NB** You are responsible to use the correct dataset for training your model ! You can make experiments by training on the `test` set or on the `test_ood_topo` set if you want but we don't recommend you do to so !

**NB** This code is generic and should work for all `AugementedSimulator`

In [ ]:
my_simulator.train(nb_iter=200,
                   train_dataset=neurips_benchmark1.train_dataset,
                   val_dataset=neurips_benchmark1.val_dataset)

And then you can save it:

In [7]:
my_simulator.save(path_save)
my_simulator.save_metadata(path_save)

##  Evaluate the augmented simulator

Or evaluate it on the test dataset as in the previous notebook:

In [8]:
metrics_per_dataset = neurips_benchmark1.evaluate_augmented_simulator(my_simulator)

A log file including some verifications is created at root directory with the name logs.log


Once saved, if you want to reuse it you can do exactly as we did in the previous notebook:
```python
relaoded_simulator = FullyConnectedAS(name="test_FullyConnectedAS")  # the name should match! The other things are loaded from the directory
relaoded_simulator.load_metadata(path_baselines)
relaoded_simulator.init()
relaoded_simulator.restore(path_baselines)
```

And you are good to go !

## LEAP nets
The leap nets allows to take into account the topology in the latent space, and have a more robust generalization performance than a simple fully connected model.

In [2]:
path_save = os.path.join("trained_models")
if not os.path.exists(path_save):
    os.mkdir(path_save)

from tensorflow.keras.layers import Dense
from lips.augmented_simulators import LeapNetAS

# the three lines bellow might be familiar to the tensorflow users. They tell tensorflow to not take all
# the GPU video RAM for the model.
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for el in physical_devices:
    tf.config.experimental.set_memory_growth(el, True)

In [3]:
leapNet = LeapNetAS(name="test_leapNetAS",
                    # `attr_x` represents the variables of the dataset you want to use to predict 
                    # the output.
                    attr_x=("prod_p", "prod_v", "load_p", "load_q"),
                    # `attr_y` represents the variables of the dataset you want to predict
                    # we predict everything needed, you can try to change them if you want, add some 
                    # others etc.
                    attr_y=("a_or", "a_ex"),
                    # `lr` is the learning rate
                    lr=3e-4, 
                    # `layer` is the type of keras layer you want to use. We don't recommend to 
                    # change it
                    layer=Dense,
                    # `layer_act` is the activation function you want to use after each layer
                    layer_act="relu",
                    # `loss` is the training loss
                    loss="mse",  # loss used to train the model
                    # `batch_size` is the size of the batch for training
                    batch_size=128)

## Train the model 

In [ ]:
my_simulator.train(nb_iter=200,
                   train_dataset=neurips_benchmark1.train_dataset,
                   val_dataset=neurips_benchmark1.val_dataset
                  )

### Evaluate the model 

In [ ]:
metrics_per_dataset = neurips_benchmark1.evaluate_augmented_simulator(my_simulator)

# Code another type of "augmented_simulator"

We provide only a single type of augmented simulator as of now. More baselines will be added with, we hope the growth of the community.

Coding another type of "augmented simulator" is not difficult. Finding one that work well for all the criteria is of course a different challenge.

Basically, an augmented simulator should:

- inherit from `AugmentedSimulator`
- implements the methods `save_metadata` and `load_metadata`
- implements the methods `save` and `restore`
- implements the method `init` that can for example, create the neural network from its meta parameters
- implements the method `train` that will train the augmented simulator for a given number of steps (it is not mandatory to make such function, some `AugmentedSimulator` might not require any training at all)
- implements the method `predict` method that will make some predictions and return a dictionnary containing the predictions for all variable types.

More information is given on the documentation. And a fully working example is given in the `FullyConnectedAS` class.

This is it, nothing more is required.